In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
# criar spark session
spark = SparkSession \
    .builder \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0') \
    .config('spark.hadoop.fs.s3a.access.key', '') \
    .config('spark.hadoop.fs.s3a.secret.key', '') \
    .appName('spark etl - pessoal inativo') \
    .getOrCreate()

# ler arquivos no s3
df = spark.read.csv('s3a://custos-stn-bucket/pessoal_inativo/pessoal_inativo_100.csv', header=True, inferSchema=True)

In [7]:
# schema
df.printSchema()

root
 |-- co_natureza_juridica: integer (nullable = true)
 |-- ds_natureza_juridica: string (nullable = true)
 |-- co_organizacao_n1: integer (nullable = true)
 |-- ds_organizacao_n1: string (nullable = true)
 |-- co_organizacao_n2: integer (nullable = true)
 |-- ds_organizacao_n2: string (nullable = true)
 |-- co_organizacao_n3: integer (nullable = true)
 |-- ds_organizacao_n3: string (nullable = true)
 |-- an_lanc: integer (nullable = true)
 |-- me_lanc: integer (nullable = true)
 |-- va_custo_pessoal_inativo: double (nullable = true)



In [8]:
# contagem de linhas
df.count()

250

In [9]:
# emprimir primeiras 5 linhas
df.show(5)

+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+--------------------+-------+-------+------------------------+
|co_natureza_juridica|ds_natureza_juridica|co_organizacao_n1|   ds_organizacao_n1|co_organizacao_n2|   ds_organizacao_n2|co_organizacao_n3|   ds_organizacao_n3|an_lanc|me_lanc|va_custo_pessoal_inativo|
+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+--------------------+-------+-------+------------------------+
|                   3|ADMINISTRACAO DIRETA|              316|MINISTERIO DA JUS...|              324|DEPARTAMENTO DE P...|             1147|SUPERINTENDENCIA ...|   2016|     12|              5081721.02|
|                   3|ADMINISTRACAO DIRETA|              316|MINISTERIO DA JUS...|              324|DEPARTAMENTO DE P...|             1147|SUPERINTENDENCIA ...|   2016|     10|              50

In [11]:
# criar view para spark sql
df.createOrReplaceTempView("pessoal_inativo")

In [12]:

df_pi = spark.sql("""
                  select *
                  from pessoal_inativo
                  """)

df_pi.printSchema()


root
 |-- co_natureza_juridica: integer (nullable = true)
 |-- ds_natureza_juridica: string (nullable = true)
 |-- co_organizacao_n1: integer (nullable = true)
 |-- ds_organizacao_n1: string (nullable = true)
 |-- co_organizacao_n2: integer (nullable = true)
 |-- ds_organizacao_n2: string (nullable = true)
 |-- co_organizacao_n3: integer (nullable = true)
 |-- ds_organizacao_n3: string (nullable = true)
 |-- an_lanc: integer (nullable = true)
 |-- me_lanc: integer (nullable = true)
 |-- va_custo_pessoal_inativo: double (nullable = true)



In [13]:
df_pi.show()

+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+--------------------+-------+-------+------------------------+
|co_natureza_juridica|ds_natureza_juridica|co_organizacao_n1|   ds_organizacao_n1|co_organizacao_n2|   ds_organizacao_n2|co_organizacao_n3|   ds_organizacao_n3|an_lanc|me_lanc|va_custo_pessoal_inativo|
+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+--------------------+-------+-------+------------------------+
|                   3|ADMINISTRACAO DIRETA|              316|MINISTERIO DA JUS...|              324|DEPARTAMENTO DE P...|             1147|SUPERINTENDENCIA ...|   2016|     12|              5081721.02|
|                   3|ADMINISTRACAO DIRETA|              316|MINISTERIO DA JUS...|              324|DEPARTAMENTO DE P...|             1147|SUPERINTENDENCIA ...|   2016|     10|              50

In [14]:
df_pi.head(1) # Mostra o cabeçalho da primeira linha

[Row(co_natureza_juridica=3, ds_natureza_juridica='ADMINISTRACAO DIRETA', co_organizacao_n1=316, ds_organizacao_n1='MINISTERIO DA JUSTICA E CIDADANIA', co_organizacao_n2=324, ds_organizacao_n2='DEPARTAMENTO DE POLICIA FEDERAL', co_organizacao_n3=1147, ds_organizacao_n3='SUPERINTENDENCIA REGIONAL DE POLICIA FEDERAL NO PARANA', an_lanc=2016, me_lanc=12, va_custo_pessoal_inativo=5081721.02)]

In [21]:
df_pi.describe(["co_natureza_juridica","ds_natureza_juridica","co_organizacao_n1","ds_organizacao_n1","co_organizacao_n2"]).show()

+-------+--------------------+--------------------+-----------------+--------------------+------------------+
|summary|co_natureza_juridica|ds_natureza_juridica|co_organizacao_n1|   ds_organizacao_n1| co_organizacao_n2|
+-------+--------------------+--------------------+-----------------+--------------------+------------------+
|  count|                 250|                 250|              250|                 250|               250|
|   mean|               3.136|                null|        29702.796|                null|          4872.712|
| stddev|  0.6253545179275268|                null|68342.58704610913|                null|14999.651330521148|
|    min|                   2|ADMINISTRACAO DIRETA|               46|ADVOCACIA-GERAL D...|                -9|
|    max|                   4|    FUNDACAO PUBLICA|           235881|MINISTERIO DOS TR...|            116538|
+-------+--------------------+--------------------+-----------------+--------------------+------------------+



In [23]:
df_pi.describe(["ds_organizacao_n2","co_organizacao_n3","ds_organizacao_n3","an_lanc","me_lanc"]).show()

+-------+--------------------+------------------+--------------------+------------------+------------------+
|summary|   ds_organizacao_n2| co_organizacao_n3|   ds_organizacao_n3|           an_lanc|           me_lanc|
+-------+--------------------+------------------+--------------------+------------------+------------------+
|  count|                 250|               250|                 250|               250|               250|
|   mean|                null|           665.252|                null|          2017.568|             6.264|
| stddev|                null|2200.3917624206183|                null|0.8627591863634767|3.4118715494402068|
|    min|AGENCIA NACIONAL ...|                -9|CENTRO DE CIENCIA...|              2015|                 1|
|    max|UNIVERSIDADE FEDE...|             13428|SUPERINTENDENCIA ...|              2019|                12|
+-------+--------------------+------------------+--------------------+------------------+------------------+



In [24]:
df_pi.describe(["va_custo_pessoal_inativo"]).show()

+-------+------------------------+
|summary|va_custo_pessoal_inativo|
+-------+------------------------+
|  count|                     250|
|   mean|       4818644.146560001|
| stddev|       9548395.388990697|
|    min|                 2516.75|
|    max|            7.33163657E7|
+-------+------------------------+



In [25]:
df_pi.select('ds_organizacao_n1').distinct().rdd.map(lambda r: r[0]).collect()
# Util para conhecer classes discretas de uma coluna categórica.
# Neste caso mostra as organizacoes presentes nestes dados

['MINISTERIO DO PLANEJAMENTO, DESENVOLVIMENTO E GESTAO',
 'MINISTERIO DA INDUSTRIA, COMERCIO EXTERIOR E SERVICOS',
 'MINISTERIO DA JUSTICA E SEGURANCA PUBLICA',
 'MINISTERIO DOS TRANSPORTES, PORTOS E AVIACAO CIVIL',
 'MINISTERIO DA SAUDE',
 'MINISTERIO DA DEFESA',
 'MINISTERIO DA TRANSPARENCIA E CONTROLADORIA-GERAL DA UNIAO',
 'MINISTERIO DO MEIO AMBIENTE',
 'MINISTERIO DA CULTURA',
 'MINISTERIO DO TRABALHO',
 'MINISTERIO DO ESPORTE',
 'MINISTERIO DA EDUCACAO',
 'MINISTERIO DE MINAS E ENERGIA',
 'MINISTERIO DA SEGURANCA PUBLICA',
 'ADVOCACIA-GERAL DA UNIAO',
 'MINISTERIO DA CIENCIA, TECNOLOGIA, INOVACOES E COMUNICACOES',
 'CONSELHO DE DEFESA NACIONAL',
 'MINISTERIO DA JUSTICA E CIDADANIA']

In [27]:
df_pi.select('ds_organizacao_n3').distinct().rdd.map(lambda r: r[0]).collect()
# Util para conhecer classes discretas de uma coluna categórica.
# Neste caso mostra as modalidades de aplicacao presentes nestes dados

['SUPERINTENDENCIA REGIONAL DE POLICIA FEDERAL NO PARANA',
 'SUPERINTENDENCIA REGIONAL DE POLICIA FEDERAL NA PARAIBA',
 'CENTRO DE CIENCIAS JURIDICAS',
 'COMANDO-GERAL DO PESSOAL',
 'CENTRO DE CIENCIAS AGRARIAS',
 'SUPERINTENDENCIA REGIONAL DA POLICIA RODOVIARIA FEDERAL NO DISTRITO FEDERAL',
 'COMANDO DE OPERACOES NAVAIS',
 'PROCURADORIA FEDERAL',
 'NAO SE APLICA',
 'COMANDO-GERAL DO CORPO DE FUZILEIROS NAVAIS']

In [28]:
df_pi = spark.sql("""
                  select distinct *
                  from pessoal_inativo
                  """)
df_pi.count() # Verificar se tem linhas duplicadas

# Retorna 250, isso quer dizer que as linhas sao todas diferentes

250

In [29]:
df_pi.head(1)

[Row(co_natureza_juridica=3, ds_natureza_juridica='ADMINISTRACAO DIRETA', co_organizacao_n1=2844, ds_organizacao_n1='MINISTERIO DO TRABALHO', co_organizacao_n2=3494, ds_organizacao_n2='SUPERINTENDENCIA REGIONAL DO TRABALHO EM MATO GROSSO DO SUL', co_organizacao_n3=-9, ds_organizacao_n3='NAO SE APLICA', an_lanc=2018, me_lanc=8, va_custo_pessoal_inativo=1047360.8)]

In [30]:
df_pi_unique = spark.sql("""
                  select  co_natureza_juridica || an_lanc || me_lanc || va_custo_pessoal_inativo ||
                    co_organizacao_n1 || co_organizacao_n2 || co_organizacao_n3, 
                  count(1)
                  from pessoal_inativo
                  group by 1
                  having count(1) > 1
                  """)
# Encontrando a chave primária através de:
# 1- co_natureza_juridica
# 2- an_lanc
# 3- me_lanc
# 4- va_custo_pessoal_inativo
# 5- co_organizacao_n1
# 6- co_organizacao_n2
# 7- co_organizacao_n3

df_pi_unique.count()

0

In [32]:
df_pi.groupBy(["co_organizacao_n3","ds_organizacao_n3"]).count().sort("count", ascending=True).show()
# Mostra a quantidade de descricao por natureza juridica

+-----------------+--------------------+-----+
|co_organizacao_n3|   ds_organizacao_n3|count|
+-----------------+--------------------+-----+
|            13428|SUPERINTENDENCIA ...|    2|
|              191|COMANDO DE OPERAC...|    2|
|            12161|CENTRO DE CIENCIA...|    3|
|            12165|CENTRO DE CIENCIA...|    3|
|               63|COMANDO-GERAL DO ...|    8|
|              354|PROCURADORIA FEDERAL|   19|
|             1153|SUPERINTENDENCIA ...|   20|
|              192|COMANDO-GERAL DO ...|   20|
|             1147|SUPERINTENDENCIA ...|   29|
|               -9|       NAO SE APLICA|  144|
+-----------------+--------------------+-----+



In [ ]:
# TODO carregar postgres